In [13]:
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
import pandas as pd

In [14]:
columns = ["Sample code number","Clump Thickness","Uniformity of Cell Size","Uniformity of Cell Shape","Marginal Adhesion","Single Epithelial Cell Size","Bare Nuclei","Bland Chromatin","Normal Nucleoli","Mitoses","Class"]
data = pd.read_csv("D:/MachineLearningCourse/Datasets/cancer.csv",names=columns,header=None)
data.head()

,Sample code number,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,1000025,5,1,1,1,2,1,3,1,1,2
1,1002945,5,4,4,5,7,10,3,2,1,2
2,1015425,3,1,1,1,2,2,3,1,1,2
3,1016277,6,8,8,1,3,4,3,7,1,2
4,1017023,4,1,1,3,2,1,3,1,1,2


In [15]:
data.drop(["Sample code number"],axis=1,inplace=True)

In [16]:
data.describe()

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bland Chromatin,Normal Nucleoli,Mitoses,Class
count,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000,699.000000
mean,4.417740,3.134478,3.207439,2.806867,3.216023,3.437768,2.866953,1.589413,2.689557
std,2.815741,3.051459,2.971913,2.855379,2.214300,2.438364,3.053634,1.715078,0.951273
min,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000
25%,2.000000,1.000000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,2.000000
50%,4.000000,1.000000,1.000000,1.000000,2.000000,3.000000,1.000000,1.000000,2.000000
75%,6.000000,5.000000,5.000000,4.000000,4.000000,5.000000,4.000000,1.000000,4.000000
max,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,4.000000


In [17]:
data.replace("?",0,inplace=True)

In [18]:
data.columns

Index(['Clump Thickness', 'Uniformity of Cell Size',
       'Uniformity of Cell Shape', 'Marginal Adhesion',
       'Single Epithelial Cell Size', 'Bare Nuclei', 'Bland Chromatin',
       'Normal Nucleoli', 'Mitoses', 'Class'],
      dtype='object')

In [19]:
X = data.iloc[:,:9]
y = data.iloc[:,9]

In [20]:
## boş olan değerleri doldurmak için
imputer = SimpleImputer()
imputedData = imputer.fit_transform(X.values)

In [21]:
scaler = MinMaxScaler()
normalizedData = scaler.fit_transform(imputedData)

## BAGGING 
Her tahminde aynı algoritmayı kullanıp ancak verisetinin farklı rastgele alt kümeleri ile eğitmek. Bu şekilde datanın farklı örnekleriyle daha toplamda daha güçlü bir model elde ederiz. Bagging yönteminde alt kümelerde aynı veriler olabilir

In [22]:
from sklearn import model_selection
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

In [23]:
X = normalizedData

10-Fold cross validation kullanıldı <br>
Bagging clasifierda estimator olarak Decision tree kullanıldı <br>
toplamda 100 ağaç eğitim setinin farklı kısımları ile eğitilicek ve model datayla feedlendiği zaman bu 100 ağacın ortaklaşa kararına göre bir tahmin bulunucak

In [24]:
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart,n_estimators=num_trees,random_state=7)
result = model_selection.cross_val_score(model,X,y,cv=10)
print(result.mean())

0.9557142857142857


# BOOSTING 
Bir kaç zayıf öğrenen birleştirilip güç bir öğrenici oluşturma amacıyla kullanılır. 
### ADABOOST
Adaboos classifier <b>stumplar</b> kullanılır. Bunlar weak learnerdır. Adaboost bir çok stumpın birleşmesi ile oluşur. Stumplar root ve 2 leaften oluşan yapılardır. <br> 
Random forestta her bir ağacın oyu eşittir. Ancak adaboostta bazı stumpların oyu daha yüksek olabilir. <b>amount of say</b>
Bir stumpın yaptığı hata diğerini etkiler. <br>
Verisetinde bulunan her örneğe bir weight veririz. Bu ilk başta 1/m olur . İlk stump için ağaçta root seçilir (gini index) ile <br>
Yanlış sınıflandırma yapılan verilerin weightleri toplanır. Bu <b> Total Error </b> olur. <br>
Bir stumpın oyunun ne olacağı <b> amount of say </b> ile bulunur. 1/2*(log((1-totalerror)/totalerro))<br>
Stumpın az hatası varsa amount of sayi artar. Error 0.5 iken amount of say 0 olur error = 1 ise amount of say - değerlere düşer ve dediğinin tersi yapılır. </b>
Yanlış sınıflandırılan verinin weigti artırırlır. new_weight = sample_weight*exp(amount_of_say) -- stumpın ne kadar kötü yada iyi olduğğuna bağlı olarak weight güncelleniyor.<br>
    Bir sonraki stumpta ağırlığı artırılan sınıflandırılmaya çalışılır

In [25]:
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_stumps = 70
kfold = model_selection.KFold(n_splits=10,random_state=seed)
model = AdaBoostClassifier(n_estimators=num_stumps)
result = model_selection.cross_val_score(model,X,y,cv=10)
print(result.mean())

C:\Users\bagat\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.9556728778467909


### Gradient Boosting 
Başlangıçta bir leaf oluşturulur ve genel tahmin yapılır. Bu tahmin regression problemlerinde ortalama bir değer olur. <br>
2.adımda ilk adımda gelen hataya göre yeni bir tree inşa edilir. <br>
Residualları hesapla predicted - actual <br>
Bundan sonra featureler kullanarak residualları tahmin etmeye çalış. (Böyle datayı çok iyi fit edeceği için ) 
Belli bir lr ile çarparak tahmin etmeye çalışıyoruz. <br>
YeniTahmin = önceki+lr*residual --- bu sayede bir önceki tahminden daha iyi bir tahmin yapıldı. <br>
Küçük adımlar ile bu ağaçlar boyunca tekrar edilir. <br>


In [26]:
from sklearn.ensemble import GradientBoostingClassifier

kfold = model_selection.KFold(n_splits=10,random_state=seed)
model = GradientBoostingClassifier(learning_rate=0.1,n_estimators=100)
result = model_selection.cross_val_score(model,X,y,cv=kfold)
print(result.mean())

C:\Users\bagat\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.9614285714285714


### XGBOOST
Gradientboosting daha hızlı ve performansı fazladır. Gradient boost sequential olduğu için yavaştır. <br>
paralel çalışır
hem lasso hem rigge regression kullanılır 

In [27]:
import xgboost

model = xgboost.XGBClassifier()

result = model_selection.cross_val_score(model,X,y,cv=10)

print(result.mean())

0.9571221532091098


# VOTING CLASSIFIER


In [104]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

kfold = model_selection.KFold(n_splits=10,random_state=seed)
estimators = []

model1 = LogisticRegression()
estimators.append(("logistic",model1))

model2 = DecisionTreeClassifier()
estimators.append(("cart",model2))

model3 = SVC()
estimators.append(('svm', model3))

ensemble = VotingClassifier(estimators)
result=model_selection.cross_val_score(ensemble,X,y,cv=kfold)
print(result.mean())

C:\Users\bagat\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.96


# Random Forest
İçerisinde bir çok decision tree bulunuyor .Bu decision treelere verisetinden baggin yöntemiyle veriler atanıyor. Bu decision treelerin her biri featurelardan seçim yapıyor ve daha sonra bu seçime göre gini hesaplanıyor. Root olarak atanıyor . Bundan sonra diğer treelerde aynı şekilde oluşturuluyori. <br>
Bu yöntem yerine bagging ile Decision tree düşünülebilir ama random forest bunun daha optimize edilmiş halidir.

In [108]:
from sklearn.ensemble import RandomForestClassifier

In [113]:
rfc = RandomForestClassifier(n_estimators=100)


In [115]:
result = model_selection.cross_val_score(rfc,X,y,cv=10)

In [116]:
result.mean()

0.9685714285714286